# Fit the diff color model for PWV



The goal is to have a model to fit on the data

- author Sylvie Dagoret-Campagne
- creation date : 2024/09/10
- update : 2024/09/28
- update : 2024/09/29 : Accept increase in number density of points

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cmx
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm
from matplotlib.gridspec import GridSpec
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os,sys
import re
import pandas as pd

from astropy.io import fits
from astropy import units as u
from astropy import constants as c

plt.rcParams["figure.figsize"] = (8,6)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

import pickle
from scipy.interpolate import RegularGridInterpolator

import scipy
from scipy.optimize import curve_fit

props = dict(boxstyle='round', facecolor='white', alpha=0.5)

In [ ]:
machine_name = os.uname().nodename
path_rubinsimphot = "repos/repos_w_2024_17/rubinsimphot/src"
if 'sdf' in machine_name:
    #machine_name_usdf = 'sdfrome001'
    print("Set environment for USDF")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)
elif 'dagoret-nb' in machine_name:
    print("Set environment for USDF Rubin Science Platform")
    newpythonpath = os.path.join(os.getenv("HOME"),path_rubinsimphot)
    sys.path.append(newpythonpath)    
elif 'mac' in machine_name:
    print("Be sure to run this notebook in conda environment named conda_py310")
else:
    print(f"Your current machine name is {machine_name}. Check your python environment")

## Config

Note the data has been generated by the notebook **SimulatePWVeffectColorsFlatSlantedSEDAuxtelData_auxtelSDSSfilters-MultiAirmass.ipynb**

In [ ]:
OBS_tag = "AUXTEL"
file0_out = f"{OBS_tag}_atmosphericzycolorgrid_params.pickle"
file1_out = f"{OBS_tag}_atmosphericzycolorgrid_datagrid.npy"

## Read files

In [ ]:
with (open(file0_out, "rb")) as openfile:
    while True:
        try:
            info_params = pickle.load(openfile)
        except EOFError:
            break


In [ ]:
info_params

In [ ]:
all_pwv = info_params['PWV']
all_zycolordiff = info_params['ZYCOLDIFF']

In [ ]:
datagrid = np.load(file1_out,mmap_mode=None, allow_pickle=False)
print(f"...... datagrid file {file1_out} read")

## Check the model

The model is in the data grid.

In [ ]:
nx = len(all_pwv)
ny = len(all_zycolordiff)
xg, yg = np.meshgrid(all_pwv, all_zycolordiff , indexing='ij')

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(3,1,figsize=(8,10))
im = ax1.imshow(datagrid,origin="lower",cmap="jet",
           extent=(all_pwv.min(),all_pwv.max(),all_zycolordiff.min(),all_zycolordiff.max()),aspect='auto')
#cbar

ax1.set_title("relative color difference : $(Z-Y)_1 - (Z-Y)_2$ ")
ax1.set_ylabel("$(Z-Y)_1-(Z-Y)_2$")
ax1.set_xlabel("pwv (mm)")
#ax1.set_aspect('auto')

ax2.plot(all_pwv, datagrid[0,:],'b-')
ax2.set_xlabel("pwv (mm)")
ax2.set_ylabel("rel. col. : $(Z-Y)_1 - (Z-Y)_2$")
ax3.plot(all_pwv, datagrid[-1,:],'r-')
ax3.set_xlabel("pwv (mm)")
ax3.set_ylabel("rel. col. : $(Z-Y)_1 - (Z-Y)_2$")

plt.tight_layout()
plt.show()

In [ ]:
interp = RegularGridInterpolator((all_pwv, all_zycolordiff), datagrid.T, bounds_error=False, fill_value=None)

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = mpl.colors.Normalize(vmin=all_zycolordiff.min(), vmax=all_zycolordiff.max())
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,6))

for z_y0 in all_zycolordiff:
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv,z_y0*np.ones(len(all_pwv))))  
    c = interp(pts)
    ax.plot(all_pwv,c,'-',c=the_color)
    
cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='Z-Y(pwv=0) (mag)')    
ax.set_xlabel("pwv (mm)")    
#ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_ylabel("$\delta C_{zy}$ (mmag)")
ax.set_title("Relative color difference Z-Y from star1,star2")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(14,8))

for idx,z_y0 in enumerate(all_zycolordiff):
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv,z_y0*np.ones(len(all_pwv))))  
    c = interp(pts)
    ax.plot(all_pwv,c,'o',lw=0.5,c=the_color)
    
    #p = np.polyfit(np.log(np.array(all_pwv)), np.log(np.abs(c)),deg=1) 
    #print(idx,z_y0,p)
    
cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='$\Delta C_{zy}(pwv=0)$ (mag)')  

ax.set_xlabel("$pwv$ (mm)")    
#ax.set_ylabel("Czy1_Czy2_abmag_rel (mmag)")
ax.set_ylabel("$\delta C_{zy}$ (mmag)")
ax.set_title("Relative color Z-Y difference between star1,star2")
ax.set_yscale('log')
ax.set_xscale('log')
ax.grid()

## The fit

The color difference $\Delta C_{ZY}(PWV)$ wrt $PWV$ is:

$
\Delta C_{ZY}(PWV) = \Delta C_{ZY}(PWV=0) + \delta C_{ZY}(PWV)
$

where:
- $\Delta C_{ZY}(PWV=0)$ is the intrinsic color difference
- $\delta C_{ZY}(PWV)$ is the color correction term with  $\delta C_{ZY}(0)=0$ by definition

$
\delta C_{ZY}(PWV) = K\cdot(PWV)^\beta
$

### Two parameters fit

Fit 

$
\delta C_{ZY}(PWV) = K\cdot(PWV)^\beta
$

or equivalently

$
\log_{10}\left( \delta C_{ZY}(PWV)\right)  = \log_{10}(K)  + \beta\cdot \log_{10} (PWV)
$

$
\log\left( \delta C_{ZY}(PWV)\right)  = \log(K)  + \beta\cdot \log (PWV)
$


In [ ]:
np.log(10)

### Show the dependence in log10

In [ ]:
FLAG_LOGDECIMAL = False

In [ ]:


fig, ax = plt.subplots(1,1,figsize=(14,8))

# containers for the parameters fitted and the conditions delta-color at PWV=0
all_p = []    # the parameters fitted
all_sel_dc = [] #delta color at PWV = 0

for idx,z_y0 in enumerate(all_zycolordiff):
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv,z_y0*np.ones(len(all_pwv))))  
    c = interp(pts)
    
    if FLAG_LOGDECIMAL:
        X = np.log10(all_pwv[1:])
    else:
        X = np.log(all_pwv[1:])
    
    
    csum = c[1:].sum()
    
    
    if csum>0 :
        if FLAG_LOGDECIMAL:
            Y = np.log10(c[1:])
        else:
            Y = np.log(c[1:])
    elif csum<0 :
        if FLAG_LOGDECIMAL:
            Y = - np.log10(-c[1:])
        else:
            Y = - np.log(-c[1:])
    else:
        Y = 0.0
        
    if csum != 0.0:    
        ax.plot(X,Y,'o',lw=0.5,c=the_color)
    
        # fit
        z = np.polyfit(X,Y,deg=1) 
        pol = np.poly1d(z)
    
        xfit = np.linspace(-2,2,50)
        yfit = pol(xfit)
    
        plt.plot(xfit,yfit,'-',c=the_color)
        all_p.append(z)
        all_sel_dc.append(z_y0)
    
cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='$\Delta C_{zy}(pwv=0)$ (mag)')  

if FLAG_LOGDECIMAL:
    ax.set_xlabel("$\log_{10}(pwv/1mm)$")    
else:
    ax.set_xlabel("$\log(pwv/1mm)$")   
if FLAG_LOGDECIMAL:
    ax.set_ylabel("$\log_{10} (\delta C_{zy}/1mmag)$")
else:
    ax.set_ylabel("$\log (\delta C_{zy}/1mmag)$")

    
    
    
if FLAG_LOGDECIMAL:    
    ax.set_title("Fit model polynom deg 1 : $\log_{10}(\delta C_{zy}) = \log_{10}(K) + \\beta \cdot \log_{10}(PWV)$ ")
else:
    ax.set_title("Fit model polynom deg 1 : $\log(\delta C_{zy}) = \log(K) + \\beta \cdot \log(PWV)$ ")

ax.grid()

ax.axvspan(-2.,-1., facecolor='grey',alpha=1)

all_sel_dc = np.array(all_sel_dc)

### Stack delta-Czy0 with fitted parameters 

In [ ]:
all_sel_dc = np.reshape(all_sel_dc, (-1, 1))
all_p = np.vstack(all_p)
fit_res = np.hstack([all_sel_dc,all_p])

In [ ]:
#fit_res

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4))

# left figure
ax1.plot(fit_res[:,0],fit_res[:,2],'r.')
ax1.set_xlabel("$\Delta C_{zy}(pwv=0)$ (mag)")
if FLAG_LOGDECIMAL: 
    ax1.set_ylabel("$\log_{10}(K/1mmag)$")
else:
    ax1.set_ylabel("$\log(K/1mmag)$")
    
# right figure
ax2.plot(fit_res[:,0],fit_res[:,1],'b.')
ax2.set_xlabel("$\Delta C_{zy}(pwv=0)$ (mag)")
ax2.set_ylabel("slope $\\beta$")

if FLAG_LOGDECIMAL:    
    suptitle = "Fit model polynom deg 1 : $\log_{10}(\delta C_{zy}) = \log_{10}(K) + \\beta \cdot \log_{10}(PWV)$ "
else:
    suptitle = "Fit model polynom deg 1 : $\log(\delta C_{zy}) = \log(K) + \\beta \cdot \log(PWV)$ "
plt.suptitle(suptitle,fontsize=14)
plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4))



# left figure
ax1.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
idxpos = np.where(fit_res[:,0]>0)[0]
idxneg = np.where(fit_res[:,0]<0)[0]
xpos = fit_res[:,0][idxpos]
ypos = fit_res[:,2][idxpos]
xneg= - fit_res[:,0][idxneg]
yneg = -fit_res[:,2][idxneg]
ax1.plot(xpos,ypos,"r-.")
ax1.plot(xneg,yneg,"r:")
    

if FLAG_LOGDECIMAL: 
    ax1.set_ylabel("$|\log_{10}(K/1mmag)|$")
else:
    ax1.set_ylabel("$|\log(K/1mmag)|$")
    
    
# right figure    
y = np.abs(fit_res[:,1])
ymean = np.mean(y)
ystd = np.std(y)
textstr = "$|\\beta| = $"+f"{ymean:.3f}" + "$\pm$" + f"{ystd:.3f}"
ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
           
ax2.plot(np.abs(fit_res[:,0]),np.abs(fit_res[:,1]),'b-.')
ax2.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax2.set_ylabel("slope $|\\beta|$")
ax2.set_ylim(0.,1.)

if FLAG_LOGDECIMAL:    
    suptitle = "Fit model polynom deg 1 : $\log_{10}(\delta C_{zy}) = \log_{10}(K) + \\beta \cdot \log_{10}(PWV)$ "
else:
    suptitle = "Fit model polynom deg 1 : $\log(\delta C_{zy}) = \log(K) + \\beta \cdot \log(PWV)$ "
plt.suptitle(suptitle,fontsize=14)
plt.tight_layout()

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,4))

# left figure
idxpos = np.where(fit_res[:,0]>0)[0]
idxneg = np.where(fit_res[:,0]<0)[0]
xpos = fit_res[:,0][idxpos]
ypos = fit_res[:,2][idxpos]
xneg=   -fit_res[:,0][idxneg]
yneg = -fit_res[:,2][idxneg]

if FLAG_LOGDECIMAL: 
    #ax1.plot(np.abs(fit_res[:,0]),np.power(10.,np.abs(fit_res[:,2])),'ro-')
    YPOS = np.power(10.,ypos)
    YNEG = np.power(10.,yneg)
    ax1.plot(xpos,np.power(10.,ypos),'r-.')
    ax1.plot(xneg,np.power(10.,yneg),'r:')
    
else:
    #ax1.plot(np.abs(fit_res[:,0]),np.exp(np.abs(fit_res[:,2])),'bo-')
    YPOS = np.exp(ypos)
    YNEG = np.exp(yneg)
    ax1.plot(xpos,np.exp(ypos),'r-.')
    ax1.plot(xneg,np.exp(yneg),'r:')
    
ax1.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")

ax1.set_ylabel("$|K|$ (mmag)")
ax1.set_ylim(0.,8.)
    
# right figure    
y = np.abs(fit_res[:,1])
ymean = np.mean(y)
ystd = np.std(y)
textstr = "$|\\beta| = $"+f"{ymean:.3f}" + "$\pm$" + f"{ystd:.3f}"
ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)

ax2.plot(np.abs(fit_res[:,0]),np.abs(fit_res[:,1]),'b-.')
ax2.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax2.set_ylabel("slope $|\\beta|$")
ax2.set_ylim(0.,1.)

if FLAG_LOGDECIMAL:    
    suptitle = "Fit model polynom deg 1 : $\log_{10}(\delta C_{zy}) = \log_{10}(K) + \\beta \cdot \log_{10}(PWV)$ "
else:
    suptitle = "Fit model polynom deg 1 : $\log(\delta C_{zy}) = \log(K) + \\beta \cdot \log(PWV)$ "
plt.suptitle(suptitle,fontsize=14)
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))

ax.plot(xpos,YPOS,'ro')
ax.plot(xneg,YNEG,'bo')

# fit
z_pos = np.polyfit(xpos,YPOS,deg=1) 
pol_pos = np.poly1d(z_pos)    
xfit_pos = np.linspace(0.,0.8,50)
yfit_pos = pol_pos(xfit_pos)

z_neg = np.polyfit(xneg,YNEG,deg=1) 
pol_neg = np.poly1d(z_neg)    
xfit_neg = np.linspace(0.,0.8,50)
yfit_neg = pol_pos(xfit_neg)
    
ax.plot(xfit_pos,yfit_pos,'-',c='r')
ax.plot(xfit_neg,yfit_neg,'-',c='b')

ax.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax.set_ylabel("$|K|$ (mmag)")

textstr = "$y = $"+f"{z_pos[0]:.3f}.x" + " + " + f"{z_pos[1]:.3f}"
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax.set_title("diff color fit withpolynom deg=1 :  K term ")

### Fit 3 parameters

Fit 

$
\delta C_{ZY}(PWV) = K\cdot(PWV)^\beta
$

or equivalently

$
\log_{10}\left( \delta C_{ZY}(PWV)\right)  = \log_{10}(K)  + \beta\cdot \log_{10} (PWV) +  \gamma \cdot \log_{10}^2 (PWV)
$

$
\log\left( \delta C_{ZY}(PWV)\right)  = \log(K)  + \beta\cdot \log (PWV) +  \gamma \cdot \log^2 (PWV)
$

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(14,8))

all_p = []
all_sel_dc = []

for idx,z_y0 in enumerate(all_zycolordiff):
    the_color = scalarMap.to_rgba(z_y0, alpha=1)
    
    pts = list(zip(all_pwv,z_y0*np.ones(len(all_pwv))))  
    c = interp(pts)
    
    if FLAG_LOGDECIMAL:
        X = np.log10(all_pwv[1:])
    else:
        X = np.log(all_pwv[1:])
    
    
    csum = c[1:].sum()
    
        
    if csum>0 :
        if FLAG_LOGDECIMAL:
            Y = np.log10(c[1:])
        else:
            Y = np.log(c[1:])
    elif csum<0 :
        if FLAG_LOGDECIMAL:
            Y = - np.log10(-c[1:])
        else:
            Y = - np.log(-c[1:])
    else:
        Y = 0.0
        
    if csum != 0.0:    
        ax.plot(X,Y,'o',lw=0.5,c=the_color)
    
        z = np.polyfit(X,Y,deg=2) 
        pol = np.poly1d(z)
    
        xfit = np.linspace(-2,2,50)
        yfit = pol(xfit)
    
        plt.plot(xfit,yfit,'-',c=the_color)
        all_p.append(z)
        all_sel_dc.append(z_y0)
    
    

cbar = fig.colorbar(scalarMap,ax=ax, orientation='vertical', label='$\Delta C_{zy}(pwv=0)$ (mag)')    

if FLAG_LOGDECIMAL:
    ax.set_xlabel("$\log_{10}(pwv/1mm)$")    
else:
    ax.set_xlabel("$\log(pwv/1mm)$")    
if FLAG_LOGDECIMAL:
    ax.set_ylabel("$\log_{10} (\delta C_{zy}/1mmag)$")
else:
    ax.set_ylabel("$\log(\delta C_{zy}/1mmag)$")
   
    
if FLAG_LOGDECIMAL:
    ax.set_title("Fit model polynom deg 2 : log10(DC) = log10(DC0) + beta(log10(PWV)) + beta((log10(PWV))^2)")
else:
    ax.set_title("Fit model polynom deg 2 : log(DC) = log(DC0) + beta(log(PWV)) + beta(log(PWV)^2)")

if FLAG_LOGDECIMAL:    
    ax.set_title("Fit model polynom deg 2 : $\log_{10}(\delta C_{zy}) = \log_{10}(K) + \\beta \cdot \log_{10}(PWV) +  \\gamma \cdot \log_{10}^2(PWV)$ ")
else:
    ax.set_title("Fit model polynom deg 2 : $\log(\delta C_{zy}) = \log(K) + \\beta \cdot \log(PWV) +  \\gamma \cdot \log^2(PWV)$ ")

ax.grid()
ax.axvspan(-2.,-1., facecolor='grey',alpha=1)

all_sel_dc = np.array(all_sel_dc)

### Stack delta-Czy0 with fitted parameters 

In [ ]:
all_sel_dc = np.reshape(all_sel_dc, (-1, 1))
all_p = np.vstack(all_p)
fit_res = np.hstack([all_sel_dc,all_p])

In [ ]:
#fit_res

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,4))


#left figure
ax1.plot(fit_res[:,0],fit_res[:,3],'r.')
ax1.set_xlabel("$\Delta C_{zy}(pwv=0)$ (mag)")
if FLAG_LOGDECIMAL: 
    ax1.set_ylabel("$\log_{10}(K/1mmag)$")
else:
    ax1.set_ylabel("$\log(K/1mmag)$")
    
    
# middle figure
ax2.plot(fit_res[:,0],fit_res[:,2],'b.')
ax2.set_xlabel("$\Delta C_{zy}(pwv=0)$ (mag)")
ax2.set_ylabel("slope $\\beta$")

# right figure
ax3.plot(fit_res[:,0],fit_res[:,1],'g.')
ax3.set_xlabel("$\Delta C_{zy}(pwv=0)$ (mag)")
ax3.set_ylabel("$\\gamma$")


if FLAG_LOGDECIMAL:    
    suptitle = "Fit model polynom deg 2 : $\log_{10}(\delta C_{zy}) = \log_{10}(K) + \\beta \cdot \log_{10}(PWV) +  \\gamma \cdot \log_{10}^2(PWV)$ "
else:
    suptitle = "Fit model polynom deg 2 : $\log(\delta C_{zy}) = \log(K) + \\beta \cdot \log(PWV) +  \\gamma \cdot \log^2(PWV)$ "
plt.suptitle(suptitle,fontsize=14)
plt.tight_layout()

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,4))

# left figure
idxpos = np.where(fit_res[:,0]>0)[0]
idxneg = np.where(fit_res[:,0]<0)[0]
xpos = fit_res[:,0][idxpos]
ypos = fit_res[:,3][idxpos]
xneg=   -fit_res[:,0][idxneg]
yneg = -fit_res[:,3][idxneg]


#ax1.plot(np.abs(fit_res[:,0]),np.abs(fit_res[:,3]),'ro')
ax1.plot(xpos,ypos,'r-.')
ax1.plot(xneg,yneg,'r:')

ax1.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
if FLAG_LOGDECIMAL: 
    ax1.set_ylabel("$|\log_{10}(K/1mmag)|$")
else:
    ax1.set_ylabel("$|\log(K/1mmag)|$")

# middle figure    
y = np.abs(fit_res[:,2])
ymean = np.mean(y)
ystd = np.std(y)
textstr = "$|\\beta| = $"+f"{ymean:.3f}" + "$\pm$" + f"{ystd:.3f}"
ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.plot(np.abs(fit_res[:,0]),np.abs(fit_res[:,2]),'b-.')
ax2.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax2.set_ylabel("slope $|\\beta|$")
ax2.set_ylim(0.,1.)


#right figure
y = np.abs(fit_res[:,1])
ymean = np.mean(y)
ystd = np.std(y)
textstr = "$|\\gamma| = $"+f"{ymean:.3f}" + "$\pm$" + f"{ystd:.3f}"
ax3.text(0.05, 0.95, textstr, transform=ax3.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax3.plot(np.abs(fit_res[:,0]),np.abs(fit_res[:,1]),'g-.')
ax3.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax3.set_ylabel("$|\\gamma|$")
ax3.set_ylim(0.,0.2)
plt.suptitle("diff color fit : polynom deg=2")

if FLAG_LOGDECIMAL:    
    suptitle = "Fit model polynom deg 2 : $\log_{10}(\delta C_{zy}) = \log_{10}(K) + \\beta \cdot \log_{10}(PWV) +  \\gamma \cdot \log_{10}^2(PWV)$ "
else:
    suptitle = "Fit model polynom deg 2 : $\log(\delta C_{zy}) = \log(K) + \\beta \cdot \log(PWV) +  \\gamma \cdot \log^2(PWV)$ "
plt.suptitle(suptitle,fontsize=14)
plt.tight_layout()

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,4))

# left figure
idxpos = np.where(fit_res[:,0]>0)[0]
idxneg = np.where(fit_res[:,0]<0)[0]
xpos = fit_res[:,0][idxpos]
ypos = fit_res[:,3][idxpos]
xneg=   -fit_res[:,0][idxneg]
yneg = -fit_res[:,3][idxneg]


if FLAG_LOGDECIMAL: 
    ax1.plot(xpos,np.power(10.,ypos),'r-.')
    ax1.plot(xneg,np.power(10.,yneg),'r:')
    YPOS = np.power(10.,ypos)
    YNEG = np.power(10.,yneg)
else:
    ax1.plot(xpos,np.exp(ypos),'r-.')
    ax1.plot(xneg,np.exp(yneg),'r:')    
    YPOS = np.exp(ypos)
    YNEG = np.exp(yneg)
ax1.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax1.set_ylabel("$|K|$ (mmag)")
ax1.set_ylim(0.,8.)

# middle figure
y = np.abs(fit_res[:,2])
ymean = np.mean(y)
ystd = np.std(y)
textstr = "$|\\beta| = $"+f"{ymean:.3f}" + "$\pm$" + f"{ystd:.3f}"
ax2.text(0.05, 0.95, textstr, transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.plot(np.abs(fit_res[:,0]),np.abs(fit_res[:,2]),'b-.')
ax2.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax2.set_ylabel("slope $|\\beta|$")
ax2.set_ylim(0.,1.)

#right figure
y = np.abs(fit_res[:,1])
ymean = np.mean(y)
ystd = np.std(y)
textstr = "$|\\gamma| = $"+f"{ymean:.3f}" + "$\pm$" + f"{ystd:.3f}"
ax3.text(0.05, 0.95, textstr, transform=ax3.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax3.plot(np.abs(fit_res[:,0]),np.abs(fit_res[:,1]),'g-.')
ax3.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax3.set_ylabel("$|\\gamma|$")
ax3.set_ylim(0.,0.2)
plt.suptitle("diff color fit : polynom deg=2")

if FLAG_LOGDECIMAL:    
    suptitle = "Fit model polynom deg 2 : $\log_{10}(\delta C_{zy}) = \log_{10}(K) + \\beta \cdot \log_{10}(PWV) +  \\gamma \cdot \log_{10}^2(PWV)$ "
else:
    suptitle = "Fit model polynom deg 2 : $\log(\delta C_{zy}) = \log(K) + \\beta \cdot \log(PWV) +  \\gamma \cdot \log^2(PWV)$ "
plt.suptitle(suptitle,fontsize=14)
plt.tight_layout()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))

ax.plot(xpos,YPOS,'ro')
ax.plot(xneg,YNEG,'bo')

# fit
z_pos = np.polyfit(xpos,YPOS,deg=1) 
pol_pos = np.poly1d(z_pos)    
xfit_pos = np.linspace(0.,0.8,50)
yfit_pos = pol_pos(xfit_pos)

z_neg = np.polyfit(xneg,YNEG,deg=1) 
pol_neg = np.poly1d(z_neg)    
xfit_neg = np.linspace(0.,0.8,50)
yfit_neg = pol_pos(xfit_neg)
    
ax.plot(xfit_pos,yfit_pos,'-',c='r')
ax.plot(xfit_neg,yfit_neg,'-',c='b')

ax.set_xlabel("$|\Delta C_{zy}(pwv=0)|$ (mag)")
ax.set_ylabel("$|K|$ (mmag)")

textstr = "$y = $"+f"{z_pos[0]:.3f}.x" + " + " + f"{z_pos[1]:.3f}"
ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax.set_title("diff color fit with polynom deg=2 : K term ")